# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../WeatherPy/output_data/weather_data.csv"
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kutum,14.2000,24.6667,82.47,20,23,6.91,SD,1666200986
1,1,Castro,-24.7911,-50.0119,73.36,71,36,6.42,BR,1666200987
2,2,Hilo,19.7297,-155.0900,80.28,82,100,4.61,US,1666200987
3,3,Busselton,-33.6500,115.3333,57.85,88,29,5.77,AU,1666200988
4,4,Avarua,-21.2078,-159.7750,73.45,78,75,9.22,CK,1666200988


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity = df["Humidity"].astype(float)

In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [6]:
# Create a humidity Heatmap layer
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Ideal weather conditions: temperature between 70 and 80 degrees; wind speed less than 5 mph; zero cloudiness
ideal_weather_df = df.loc[(df['Max Temp'] > 70) & (df['Max Temp'] < 80) & (df['Wind Speed'] < 5) 
                          & (df['Cloudiness'] == 0), 
                          ['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
117,‘Ewa Beach,21.3156,-158.0072,79.02,75,0,4.61,US,1666201055
140,Karīmganj,24.8667,92.3500,71.19,83,0,3.06,IN,1666201065
213,Makakilo City,21.3469,-158.0858,77.61,75,0,4.61,US,1666201099
273,Kahului,20.8947,-156.4700,79.84,73,0,1.01,US,1666200933
421,Saint George,37.1041,-113.5841,71.44,27,0,3.44,US,1666201008
452,Arrondissement d'Arcachon,44.5875,-1.0519,73.72,78,0,1.14,FR,1666201258
482,Osorno,-40.5667,-73.1500,70.25,59,0,2.30,CL,1666201281
549,Poggiomarino,40.8031,14.5390,70.70,90,0,0.13,IT,1666201318
565,Gujan-Mestras,44.6333,-1.0667,72.07,78,0,2.30,FR,1666201331


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_weather_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,‘Ewa Beach,21.3156,-158.0072,79.02,75,0,4.61,US,1666201055
1,Karīmganj,24.8667,92.3500,71.19,83,0,3.06,IN,1666201065
2,Makakilo City,21.3469,-158.0858,77.61,75,0,4.61,US,1666201099
3,Kahului,20.8947,-156.4700,79.84,73,0,1.01,US,1666200933
4,Saint George,37.1041,-113.5841,71.44,27,0,3.44,US,1666201008
5,Arrondissement d'Arcachon,44.5875,-1.0519,73.72,78,0,1.14,FR,1666201258
6,Osorno,-40.5667,-73.1500,70.25,59,0,2.30,CL,1666201281
7,Poggiomarino,40.8031,14.5390,70.70,90,0,0.13,IT,1666201318
8,Gujan-Mestras,44.6333,-1.0667,72.07,78,0,2.30,FR,1666201331


In [9]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,‘Ewa Beach,21.3156,-158.0072,79.02,75,0,4.61,US,1666201055,
1,Karīmganj,24.8667,92.3500,71.19,83,0,3.06,IN,1666201065,
2,Makakilo City,21.3469,-158.0858,77.61,75,0,4.61,US,1666201099,
3,Kahului,20.8947,-156.4700,79.84,73,0,1.01,US,1666200933,
4,Saint George,37.1041,-113.5841,71.44,27,0,3.44,US,1666201008,
5,Arrondissement d'Arcachon,44.5875,-1.0519,73.72,78,0,1.14,FR,1666201258,
6,Osorno,-40.5667,-73.1500,70.25,59,0,2.30,CL,1666201281,
7,Poggiomarino,40.8031,14.5390,70.70,90,0,0.13,IT,1666201318,
8,Gujan-Mestras,44.6333,-1.0667,72.07,78,0,2.30,FR,1666201331,


In [10]:
# params dictionary to update each iteration
params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # base url 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request 
    response = requests.get(base_url, params=params).json()
    hotel = response['results']
    # pprint 
    # pprint(hotel)

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel[0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [11]:
# hotel_df[['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date', 'Hotel Name']]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,‘Ewa Beach,21.3156,-158.0072,79.02,75,0,4.61,US,1666201055,Base Lodging
1,Karīmganj,24.8667,92.3500,71.19,83,0,3.06,IN,1666201065,AHAR /AHELI HOTEL & RESTAURANT
2,Makakilo City,21.3469,-158.0858,77.61,75,0,4.61,US,1666201099,Marriott's Ko Olina Beach Club
3,Kahului,20.8947,-156.4700,79.84,73,0,1.01,US,1666200933,Maui Seaside Hotel
4,Saint George,37.1041,-113.5841,71.44,27,0,3.44,US,1666201008,Best Western Plus Abbey Inn
5,Arrondissement d'Arcachon,44.5875,-1.0519,73.72,78,0,1.14,FR,1666201258,Hôtel All Suites Le Teich
6,Osorno,-40.5667,-73.1500,70.25,59,0,2.30,CL,1666201281,Sonesta Hotel Osorno
7,Poggiomarino,40.8031,14.5390,70.70,90,0,0.13,IT,1666201318,Hotel la Fenice
8,Gujan-Mestras,44.6333,-1.0667,72.07,78,0,2.30,FR,1666201331,Hôtel All Suites Le Teich


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Convert Hotel Name as a list
names = hotel_df["Hotel Name"].tolist()
# Convert City to list
cities = hotel_df["City"].tolist()
# Convert Country to list
countries = hotel_df["Country"].tolist()
# Combine lists
list_of_lists = [names, cities, countries]

# [(i) for sublist in list_of_lists for i in sublist]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
                            
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…